In [1]:
#import os
#import gradio as gr
import numpy as np
import torch
from mobile_sam import SamAutomaticMaskGenerator, SamPredictor, sam_model_registry
# from PIL import ImageDraw
# from mobile_sam.utils.tools import box_prompt, format_results, point_prompt
from app.utils.tools_gradio import fast_process
# Most of our demo code is from [FastSAM Demo](https://huggingface.co/spaces/An-619/FastSAM). Huge thanks for AN-619.

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device


c:\Users\newma\AppData\Local\Programs\Python\Python310\lib\site-packages\timm\models\layers\__init__.py:48: FutureWarning: Importing from timm.models.layers is deprecated, please import via timm.layers
  warnings.warn(f"Importing from {__name__} is deprecated, please import via timm.layers", FutureWarning)
c:\Users\newma\AppData\Local\Programs\Python\Python310\lib\site-packages\timm\models\registry.py:4: FutureWarning: Importing from timm.models.registry is deprecated, please import via timm.models
  warnings.warn(f"Importing from {__name__} is deprecated, please import via timm.models", FutureWarning)
c:\KU\ATIA\MobileSAM\mobile_sam\modeling\tiny_vit_sam.py:656: UserWarning: Overwriting tiny_vit_5m_224 in registry with mobile_sam.modeling.tiny_vit_sam.tiny_vit_5m_224. This is because the name being registered conflicts with an existing name. Please check if this is not expected.
  return register_model(fn_wrapper)
c:\KU\ATIA\MobileSAM\mobile_sam\modeling\tiny_vit_sam.py:656: UserWarni

device(type='cpu')

In [ ]:
# Load the pre-trained model
sam_checkpoint = "weights\mobile_sam.pt"
# Define the model type: Tiny Vit
model_type = "vit_t"
# Load the model
mobile_sam = sam_model_registry[model_type](checkpoint=sam_checkpoint)
# Move the model to the device
mobile_sam = mobile_sam.to(device=device)
# Set the model to evaluation mode
mobile_sam.eval()

In [3]:
from PIL import Image
image = Image.open('app/assets/picture7.jpg')
#image = np.asarray(image)

In [5]:
from transformers import SamProcessor

In [6]:
processor = SamProcessor.from_pretrained("facebook/sam-vit-huge")

In [32]:
from PIL import Image
image = Image.open('app/assets/picture7.jpg').convert("RGB")
image = np.asarray(image)
predictor.set_image(image)

In [37]:
#from segment_anything import SamPredictor, sam_model_registry
from models.sam import SamPredictor, sam_model_registry
from models.sam.utils.transforms import ResizeLongestSide
from skimage.measure import label
from models.sam_LoRa import LoRA_Sam
#Scientific computing 
import numpy as np
import os
#Pytorch packages
import torch
from torch import nn
import torch.optim as optim
import torchvision
from torchvision import datasets
from tensorboardX import SummaryWriter
#Visulization
import matplotlib.pyplot as plt
from torchvision import transforms
from PIL import Image
#Others
from torch.utils.data import DataLoader, Subset
from torch.autograd import Variable
import matplotlib.pyplot as plt
import copy
from utils.dataset import Public_dataset
import torch.nn.functional as F
from torch.nn.functional import one_hot
from pathlib import Path
from tqdm import tqdm
from utils.losses import DiceLoss
from utils.dsc import dice_coeff_multi_class
import cv2
import monai
from utils.utils import vis_image
import cfg
import json

ModuleNotFoundError: No module named 'models'

In [36]:
#def train_model(trainloader,valloader,dir_checkpoint,epochs):
# if args.if_warmup:
#     b_lr = args.lr / args.warmup_period
# else:
#     b_lr = args.lr

#sam = sam_model_registry[args.arch](args,checkpoint=os.path.join(args.sam_ckpt),num_classes=args.num_cls)
# if args.finetune_type == 'adapter':
#     for n, value in sam.named_parameters():
#         if "Adapter" not in n: # only update parameters in adapter
#             value.requires_grad = False
#     print('if update encoder:',args.if_update_encoder)
#     print('if image encoder adapter:',args.if_encoder_adapter)
#     print('if mask decoder adapter:',args.if_mask_decoder_adapter)
#     if args.if_encoder_adapter:
#         print('added adapter layers:',args.encoder_adapter_depths)
    
# elif args.finetune_type == 'vanilla' and args.if_update_encoder==False:   
#     print('if update encoder:',args.if_update_encoder)
#     for n, value in sam.image_encoder.named_parameters():
#         value.requires_grad = False
# elif args.finetune_type == 'lora':
#     print('if update encoder:',args.if_update_encoder)
#     print('if image encoder lora:',args.if_encoder_lora_layer)
#     print('if mask decoder lora:',args.if_decoder_lora_layer)
#     sam = LoRA_Sam(args,sam,r=4).sam
# sam.to('cuda')
    
optimizer = optim.AdamW(sam.parameters(), lr=b_lr, betas=(0.9, 0.999), eps=1e-08, weight_decay=0.1, amsgrad=False)
optimizer.zero_grad()
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=10, gamma=0.5) #learning rate decay
criterion1 = monai.losses.DiceLoss(sigmoid=True, squared_pred=True, to_onehot_y=True,reduction='mean')
criterion2 = nn.CrossEntropyLoss()

iter_num = 0
max_iterations = epochs * len(trainloader) 
writer = SummaryWriter(dir_checkpoint + '/log')

pbar = tqdm(range(epochs))
val_largest_dsc = 0
last_update_epoch = 0
for epoch in pbar:
    sam.train()
    train_loss = 0
    for i,data in enumerate(tqdm(trainloader)):
        imgs = data['image'].cuda()
        msks = torchvision.transforms.Resize((args.out_size,args.out_size))(data['mask'])
        msks = msks.cuda()

        if args.if_update_encoder:
            img_emb = sam.image_encoder(imgs)
        else:
            with torch.no_grad():
                img_emb = sam.image_encoder(imgs)
        
        # get default embeddings
        sparse_emb, dense_emb = sam.prompt_encoder(
            points=None,
            boxes=None,
            masks=None,
        )
        pred, _ = sam.mask_decoder(
                        image_embeddings=img_emb,
                        image_pe=sam.prompt_encoder.get_dense_pe(), 
                        sparse_prompt_embeddings=sparse_emb,
                        dense_prompt_embeddings=dense_emb, 
                        multimask_output=True,
                        )
        loss_dice = criterion1(pred,msks.float()) 
        loss_ce = criterion2(pred,torch.squeeze(msks.long(),1))
        loss =  loss_dice + loss_ce
        
        loss.backward()
        optimizer.step()
        optimizer.zero_grad(set_to_none=True)
        
        if args.if_warmup and iter_num < args.warmup_period:
            lr_ = args.lr * ((iter_num + 1) / args.warmup_period)
            for param_group in optimizer.param_groups:
                param_group['lr'] = lr_

        else:
            if args.if_warmup:
                shift_iter = iter_num - args.warmup_period
                assert shift_iter >= 0, f'Shift iter is {shift_iter}, smaller than zero'
                lr_ = args.lr * (1.0 - shift_iter / max_iterations) ** 0.9  # learning rate adjustment depends on the max iterations
                for param_group in optimizer.param_groups:
                    param_group['lr'] = lr_
            else:
                lr_ = args.lr

        train_loss += loss.item()
        iter_num+=1
        writer.add_scalar('info/lr', lr_, iter_num)
        writer.add_scalar('info/total_loss', loss, iter_num)
        writer.add_scalar('info/loss_ce', loss_ce, iter_num)
        writer.add_scalar('info/loss_dice', loss_dice, iter_num)

    train_loss /= (i+1)
    pbar.set_description('Epoch num {}| train loss {} \n'.format(epoch,train_loss))

NameError: name 'optim' is not defined

torch.Size([3, 1024, 1024])

ValueError: expected 4D input (got 3D input)